In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
from math import sqrt
from numpy import split
from numpy import array
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from sklearn import preprocessing

In [ ]:
train = read_csv('../input/covid19-global-forecasting-week-2/train.csv', header=0, infer_datetime_format=True, parse_dates=['Date'], index_col=['Date'])
test = read_csv('../input/covid19-global-forecasting-week-2/test.csv', header=0, infer_datetime_format=True, parse_dates=['Date'], index_col=['Date'])
submission = read_csv('../input/covid19-global-forecasting-week-2/submission.csv')

In [ ]:
train.drop(["Id", "Province_State", "Fatalities"], axis=1, inplace=True)
test.drop(["ForecastId", "Province_State"], axis=1, inplace=True)

In [ ]:
train["date"] = train.index
test["date"] = test.index

In [ ]:
train

In [ ]:
test

Add in more data

In [ ]:
hdi = pd.read_csv("../input/world-bank-datasets/Human_Development_Index.csv")

In [ ]:
train = pd.merge(left=train, right=hdi, how='left', left_on='Country_Region', right_on='Country')
test = pd.merge(left=test, right=hdi, how='left', left_on='Country_Region', right_on='Country')
train.drop(['Country', 'Gross national income (GNI) per capita 2018'],axis=1, inplace=True)
test.drop(['Country', 'Gross national income (GNI) per capita 2018'],axis=1, inplace=True)

In [ ]:
train.info()

In [ ]:
test

In [ ]:
#train = pd.concat([train,pd.get_dummies(train['Country_Region'], prefix='cr')],axis=1)
#train.drop(['Country_Region'],axis=1, inplace=True)
#test = pd.concat([test,pd.get_dummies(test['Country_Region'], prefix='cr')],axis=1)
#test.drop(['Country_Region'],axis=1, inplace=True)
#le = preprocessing.LabelEncoder()
#le.fit(train['Country_Region'])
#train['Country_Region'] = le.transform(train['Country_Region']) 
#test['Country_Region'] = le.transform(test['Country_Region'])

In [ ]:
train

In [ ]:
test

In [ ]:
train.fillna(0, inplace=True)
test.fillna(0, inplace=True)

In [ ]:
train.drop(['Country_Region'],axis=1, inplace=True)
test.drop(['Country_Region'],axis=1, inplace=True)

In [ ]:
train

In [ ]:
test

In [ ]:
test['ConfirmedCases'] = 0.0

In [ ]:
test

In [ ]:
cols = test.columns.tolist()
cols = cols[-1:] + cols[:-1]
test = test[cols]

In [ ]:
test

In [ ]:
basedate = pd.Timestamp('2020-01-01')
train['time since'] = train.apply(lambda x: (x.date - basedate).days, axis=1)
test['time since'] = test.apply(lambda x: (x.date - basedate).days, axis=1)

In [ ]:
train

In [ ]:
test

In [ ]:
train.drop("date", axis=1, inplace=True)
test.drop("date", axis=1, inplace=True)

In [ ]:
train

In [ ]:
test

In [ ]:
def split_dataset(data):
    train, test = data[:-328], data[-328:]
    train = array(split(train, len(train)))
    test = array(split(test, len(test)))
    return train, test

In [ ]:
def evaluate_forecasts(actual, predicted):
    scores = list()
    for i in range(actual.shape[1]):
        mse = mean_squared_error(actual[:,i], predicted[:,i])
        rmse = sqrt(mse)
        scores.append(rmse)
    s = 0
    for row in range(actual.shape[0]):
        for col in range(actual.shape[1]):
            s += (actual[row,col] - predicted[row,col])**2
    score = sqrt(s/(actual.shape[0]*actual.shape[1]))
    return score, scores

In [ ]:
def summarize_scores(name, score, scores):
    s_scores = ", ".join(["%.1f" % s for s in scores])
    print("%s: [%.3f] %s" %(name, score, s_scores))

In [ ]:
def to_supervised(train, n_input, n_out=1):
    data = train.reshape((train.shape[0]*train.shape[1], train.shape[2]))
    X, y = list(), list()
    in_start = 0
    for _ in range(len(data)):
        in_end = in_start + n_input
        out_end = in_end + n_out
        if out_end <= len(data):
            X.append(data[in_start:in_end, :])
            y.append(data[in_end:out_end, 0])
        in_start += 1
    return array(X), array(y)

In [ ]:
def build_model(train, n_input):
    train_x, train_y = to_supervised(train, n_input)
    verbose, epochs, batch_size = 0, 140, 16
    n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]
    model = Sequential()
    model.add(Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=(n_timesteps, n_features)))
    model.add(MaxPooling1D(pool_size=1))
    model.add(Conv1D(filters=16, kernel_size=2, activation='relu'))
    model.add(MaxPooling1D(pool_size=1))
    model.add(Conv1D(filters=8, kernel_size=2, activation='relu'))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs))
    model.compile(loss='mse', optimizer='adam')
    model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, verbose=verbose)
    return model

In [ ]:
def forecast(model, history, n_input):
    data = array(history)
    data = data.reshape((data.shape[0]*data.shape[1], data.shape[2]))
    input_x = data[-n_input:,:]
    input_x = input_x.reshape((1, input_x.shape[0], input_x.shape[1]))
    yhat = model.predict(input_x, verbose=0)
    yhat = yhat[0]
    if yhat < 0.0:
        yhat = array(0.0)
    print(yhat)
    return yhat

In [ ]:
def evaluate_model(train, test, n_input):
    model = build_model(train, n_input)
    history = [x for x in train]
    predictions = list()
    for i in range(len(test)):
        yhat_sequence = forecast(model, history, n_input)
        predictions.append(yhat_sequence)
        history.append(test[i,:])
    predictions = array(predictions)
    #score, scores = evaluate_forecasts(test[:,:,0], predictions)
    return predictions

In [ ]:
n_input = 6

In [ ]:
train.info()

In [ ]:
train_v = train.values
test_v = test.values

In [ ]:
train_v

In [ ]:
train_v = array(split(train_v, len(train)))
test_v = array(split(test_v, len(test)))

In [ ]:
len(train_v)

In [ ]:
test_v

In [ ]:
predictionsConfirmedCases = evaluate_model(train_v, test_v, n_input)

In [ ]:
submission["ConfirmedCases"] = predictionsConfirmedCases.astype(int)

In [ ]:
submission

In [ ]:
train = read_csv('../input/covid19-global-forecasting-week-2/train.csv', header=0, infer_datetime_format=True, parse_dates=['Date'], index_col=['Date'])
test = read_csv('../input/covid19-global-forecasting-week-2/test.csv', header=0, infer_datetime_format=True, parse_dates=['Date'], index_col=['Date'])

In [ ]:
train.drop(["Id", "Province_State", "ConfirmedCases"], axis=1, inplace=True)
test.drop(["ForecastId", "Province_State"], axis=1, inplace=True)

In [ ]:
train = pd.merge(left=train, right=hdi, how='left', left_on='Country_Region', right_on='Country')
test = pd.merge(left=test, right=hdi, how='left', left_on='Country_Region', right_on='Country')
train.drop(['Country', 'Gross national income (GNI) per capita 2018'],axis=1, inplace=True)
test.drop(['Country', 'Gross national income (GNI) per capita 2018'],axis=1, inplace=True)

In [ ]:
#train = pd.concat([train,pd.get_dummies(train['Country_Region'], prefix='cr')],axis=1)
train.drop(['Country_Region'],axis=1, inplace=True)
#test = pd.concat([test,pd.get_dummies(test['Country_Region'], prefix='cr')],axis=1)
test.drop(['Country_Region'],axis=1, inplace=True)

In [ ]:
train.fillna(0, inplace=True)
test.fillna(0, inplace=True)

In [ ]:
test['Fatalities'] = 0.0

In [ ]:
cols = test.columns.tolist()
cols = cols[-1:] + cols[:-1]
test = test[cols]

In [ ]:
basedate = pd.Timestamp('2020-01-01')
train['time since'] = train.apply(lambda x: (x.date - basedate).days, axis=1)
test['time since'] = test.apply(lambda x: (x.date - basedate).days, axis=1)

In [ ]:
train.drop("date", axis=1, inplace=True)
test.drop("date", axis=1, inplace=True)

In [ ]:
train

In [ ]:
test

In [ ]:
train_v = train.values
test_v = test.values

In [ ]:
train_v = array(split(train_v, len(train)))
test_v = array(split(test_v, len(test)))

In [ ]:
predictionsFatalities = evaluate_model(train_v, test_v, n_input)

In [ ]:
submission["Fatalities"] = predictionsFatalities.astype(int)

In [ ]:
submission

In [ ]:
submission.to_csv('submission.csv', index=False)